In [1]:
import os

In [2]:
%pwd

'c:\\Users\\rahul\\Desktop\\Project\\Text-Summarizer-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\rahul\\Desktop\\Project\\Text-Summarizer-Project'

##### Entity

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: str
    gradient_accumulation_steps: int

##### Configuration Manager

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.eval_steps,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )

        return model_trainer_config

##### Components

In [8]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

c:\Users\rahul\Desktop\Project\Text-Summarizer-Project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-11-03 16:59:18,561: INFO: config: PyTorch version 2.5.1 available.]


In [9]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model = model_pegasus)

        # Loading the data
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        # Defining the training arguments
        trainer_args = TrainingArguments(
            output_dir = self.config.root_dir, num_train_epochs = self.config.num_train_epochs, warmup_steps = self.config.warmup_steps,
            per_device_train_batch_size = self.config.per_device_train_batch_size, per_device_eval_batch_size = self.config.per_device_train_batch_size,
            weight_decay = self.config.weight_decay, logging_steps = self.config.logging_steps,
            evaluation_strategy = self.config.evaluation_strategy, eval_steps = self.config.eval_steps, save_steps = 1e6,
            gradient_accumulation_steps = self.config.gradient_accumulation_steps
        )

        trainer = Trainer(model = model_pegasus, args = trainer_args,
                          tokenizer = tokenizer, data_collator = seq2seq_data_collator,
                          train_dataset = dataset_samsum_pt["test"],
                          eval_dataset = dataset_samsum_pt["validation"])
        
        trainer.train()
        
        # Save the model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir, "pegasus-samsum-model"))

        # Save the tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))

##### Pipeline

In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2024-11-03 16:59:18,946: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-11-03 16:59:18,950: INFO: common: yaml file: params.yaml loaded successfully]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\rahul\Desktop\Project\Text-Summarizer-Project\.venv\Lib\site-packages\transformers\training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\rahul\AppData\Local\Temp\ipykernel_56788\739487992.py:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model = model_pegasus, args = trainer_args,
 16%|█▌        | 8/51 [04:43<25:27, 35.52s/it]c:\Users\rahul\Desktop\Project\Text-Summarizer-Project\.venv\Lib\site-package

{'loss': 49.7609, 'grad_norm': 510.97784423828125, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.2}


 39%|███▉      | 20/51 [12:05<18:07, 35.07s/it]

{'loss': 48.7465, 'grad_norm': 253.49427795410156, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.39}


...

{'loss': 47.8407, 'grad_norm': 328.2646789550781, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.78}


 98%|█████████▊| 50/51 [29:11<00:34, 34.80s/it]

{'loss': 45.6611, 'grad_norm': 784.6051635742188, 'learning_rate': 5e-06, 'epoch': 0.98}


100%|██████████| 51/51 [29:45<00:00, 34.64s/it]c:\Users\rahul\Desktop\Project\Text-Summarizer-Project\.venv\Lib\site-packages\transformers\modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
100%|██████████| 51/51 [29:54<00:00, 35.18s/it]


{'train_runtime': 1794.2213, 'train_samples_per_second': 0.456, 'train_steps_per_second': 0.028, 'train_loss': 48.370045531029795, 'epoch': 1.0}
